In [1]:
import sys
# pyberny
sys.path.append('/home/jsmith48/Gits/pyberny')
from berny import optimize, geomlib

from ase_interface import ANI, ANIENS
from ase_interface import ensemblemolecule

import numpy as np
import time
# ASE
import  ase
from ase import Atoms
from ase.io import read, write

import os
import tempfile
import subprocess
import shutil

In [2]:
geom_file = '/home/jsmith48/scratch/extensibility_test_sets/COMP6v1/ani_md_benchmark/split_dir/TrpCage1L2Y_crd.xyz'
mol = geomlib.readfile(geom_file)

In [3]:
# Set locations of all required network files
wkdir = '/home/jsmith48/scratch/ccsd_extrapolation/ccsd_train/tl_train_dhl_10/' #
cnstfile = wkdir + 'rHCNO-5.2R_16-3.5A_a4-8.params' # AEV parameters
saefile  = wkdir + 'sae_linfit.dat' # Atomic shifts
nnfdir   = wkdir + '/train' # network prefix
Nn = 4 # Number of networks in the ensemble
gpu = 5 # GPU ID -- this defaults to 0

# Initilize network ensemble
aens = ensemblemolecule(cnstfile, saefile, nnfdir, Nn, gpu)

def ANISolver(aens):
    kcal, ev, angstrom = 627.509474, 27.21138602, 0.52917721067
    #tmpdir = tempfile.mkdtemp()
    try:
        atoms = yield
        while True:
            #list of coords
            X = []
            #list of elements
            S = []
            for el, coord in atoms:
                S.append(el)
                X.append(coord)
            
            #construct ASE molecule
            geometry = Atoms(S,X)
            geometry.set_calculator(ANIENS(aens))
            
            energy = geometry.get_potential_energy()/ev
            gradients = -geometry.get_forces()/ev*angstrom

            atoms = yield energy, gradients
    finally:
        pass


In [4]:
# default params in ypberny
params = {'gradientmax': 0.45e-3,
    'gradientrms': 0.3e-3,
    'stepmax': 1.8e-3,
    'steprms': 1.2e-3,
    'trust': 0.3}

In [5]:
%%time
# define solver i.e. energy potentials
solver = ANISolver(aens)
#
# set verbosity=-1 for less info 
final = optimize(solver, mol, maxsteps=250, verbosity=0, prams=params)
print(' ')

0 Internal coordinates:
0 * Number of fragments: 1
0 * Number of internal coordinates: 1751
0 * Number of strong bonds: 318
0 * Number of strong angles: 576
0 * Number of strong dihedrals: 857
1 Energy: -7648.65807608


KeyboardInterrupt: 

In [6]:
#create final ASE molecule
#list of coords
X = []
#list of elements
S = []
for el, coord in mol:
    S.append(el)
    X.append(coord)

#construct ASE molecule
geometry = Atoms(S,X)

In [7]:
#save file
write('taxol_opt.xyz', geometry)